In [1]:
import numpy as np
import warnings
import copy
import os
import pandas as pd
warnings.filterwarnings("ignore")
import sys
sys.path.append('../induction_motor_anomaly_detection/')
import modules,scaler

In [2]:
normal_data = pd.read_csv('../data/combined_data.csv')
anomalous_data = pd.read_csv('../anomalous_data/anomalous_data.csv')

In [3]:
train_data_features = modules.ElectricalFeatureExtractor(current_data=normal_data)
test_data_features = modules.ElectricalFeatureExtractor(current_data=anomalous_data)
# train_scaler = scaler.Scaler()
# train_scaler.fit_unlabelled_data(train_data_features.feature_dataframe[0])
# train_scaled = train_scaler.transform(train_data_features.feature_dataframe[0])
# test_scaled = train_scaler.transform(test_data_features.feature_dataframe[0])

In [59]:
import numpy as np
from scipy.stats import multivariate_normal

class AnomalyDetector:
    def __init__(self, threshold=0.01):
        self.mean = None
        self.std = None
        self.threshold = threshold

    def fit(self, X):
        """
        Calculate the mean and standard deviation of each feature in the dataset
        """
        self.mean = np.mean(X, axis=0)
        self.std = np.std(X, axis=0)

    def detect_zscore(self, X):
        """
        Detect outliers using Z-score
        """
        z_scores = np.abs((X - self.mean) / self.std)
        threshold = z_scores.mean() + 3 * z_scores.std()
        indices = np.unique(np.where(z_scores > threshold)[0])
        return indices

    def detect_pdf(self, X):
        """
        Detect outliers using PDF
        """
        pdf = multivariate_normal.pdf(X, mean=self.mean, cov=np.diag(self.std ** 2))
        # is_anomaly = pdf < self.threshold
        return np.unique(np.where(pdf < self.threshold))


In [7]:
train = train_data_features.feature_dataframe[0]

In [12]:
train.isna().sum().sum()

0

In [60]:
anomaly_detector = AnomalyDetector()
anomaly_detector.fit(train)

In [61]:
anomaly_detector.detect_zscore(train)

array([229, 230, 231, 232, 234, 235, 239, 242, 246, 254, 271, 284],
      dtype=int64)

In [62]:
anomaly_detector.detect_pdf(train)


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [58]:
anomaly_detector.detect_pdf(test_data_features.feature_dataframe[0])

array([], dtype=int64)

array([229, 230, 231, 232, 234, 235, 239, 242, 246, 254, 271, 284],
      dtype=int64)